This is a simple implementation of the 2nd place code by https://github.com/baosenguo/Kaggle-MoA-2nd-Place-Solution/blob/main/training/1d-cnn-train.ipynb. This code deletes seed and kfold. Just go through the data preprocessing and the model.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install iterative-stratification

In [ ]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
from copy import deepcopy as dp
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def norm_fit(df_1,saveM = True, sc_name = 'zsco'):   
    from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler,RobustScaler,Normalizer,QuantileTransformer,PowerTransformer
    ss_1_dic = {'zsco':StandardScaler(),
                'mima':MinMaxScaler(),
                'maxb':MaxAbsScaler(), 
                'robu':RobustScaler(),
                'norm':Normalizer(), 
                'quan':QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal"),
                'powe':PowerTransformer()}
    ss_1 = ss_1_dic[sc_name]
    df_2 = pd.DataFrame(ss_1.fit_transform(df_1),index = df_1.index,columns = df_1.columns)
    if saveM == False:
        return(df_2)
    else:
        return(df_2,ss_1)

In [ ]:
def norm_tra(df_1,ss_x):
    df_2 = pd.DataFrame(ss_x.transform(df_1),index = df_1.index,columns = df_1.columns)
    return(df_2)

In [ ]:
def g_table(list1):
    table_dic = {}
    for i in list1:
        if i not in table_dic.keys():
            table_dic[i] = 1
        else:
            table_dic[i] += 1
    return(table_dic)

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
seed_everything(seed=42)

SEED = [0, 1, 2, 3 ,4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
input_dir = '../input/lish-moa/'

In [ ]:
sc_dic = {}
feat_dic = {}
train_features = pd.read_csv(input_dir+'train_features.csv')
train_targets_scored = pd.read_csv(input_dir+'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(input_dir+'train_targets_nonscored.csv')
test_features = pd.read_csv(input_dir+'test_features.csv')
sample_submission = pd.read_csv(input_dir+'sample_submission.csv')
train_drug = pd.read_csv(input_dir+'train_drug.csv')

target_cols = train_targets_scored.drop('sig_id', axis=1).columns.values.tolist()
target_nonsc_cols = train_targets_nonscored.drop('sig_id', axis=1).columns.values.tolist()

In [ ]:
nonctr_id = train_features.loc[train_features['cp_type']!='ctl_vehicle','sig_id'].tolist()
tmp_con1 = [i in nonctr_id for i in train_targets_scored['sig_id']]

In [ ]:
mat_cor = pd.DataFrame(
    np.corrcoef(
        train_targets_scored.drop('sig_id',axis = 1)[tmp_con1].T,
        train_targets_nonscored.drop('sig_id',axis = 1)[tmp_con1].T
    )
)

In [ ]:
mat_cor

In [ ]:
mat_cor2 = mat_cor.iloc[(train_targets_scored.shape[1]-1):,0:train_targets_scored.shape[1]-1]

In [ ]:
mat_cor2

In [ ]:
mat_cor2.index = target_nonsc_cols
mat_cor2.columns = target_cols
mat_cor2 = mat_cor2.dropna()

In [ ]:
mat_cor2

In [ ]:
mat_cor2_max = mat_cor2.abs().max(axis = 1)

In [ ]:
q_n_cut = 0.9
target_nonsc_cols2 = mat_cor2_max[mat_cor2_max > np.quantile(mat_cor2_max,q_n_cut)].index.tolist()

In [ ]:
print(len(target_nonsc_cols2))

In [ ]:
feat_dic = {}
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
feat_dic['gene'] = GENES
feat_dic['cell'] = CELLS

In [ ]:
def normalization(df,feature, iscell = False):
    q2 = df[feat_dic[feature]].apply(np.quantile,axis = 1,q = 0.25).copy()
    q7 = df[feat_dic[feature]].apply(np.quantile,axis = 1,q = 0.75).copy()
    qmean = (q2+q7)/2
    df[feat_dic[feature]] = (df[feat_dic[feature]].T - qmean.values).T
    if iscell == True:
        qmean2 = df[feat_dic[feature].abs().apply(np.quantile,axis = 1,q = 0.75).copy()+4
        df[feat_dic[feature]] = (df[feat_dic[feature]].T / qmean2.values).T.copy()
    return df

In [ ]:
feature_cols = []
for key_i in feat_dic.keys():
    value_i = feat_dic[key_i]
    print(key_i,len(value_i))
    feature_cols += value_i
len(feature_cols)

In [ ]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored[['sig_id']+target_nonsc_cols2], on='sig_id')

In [ ]:
train

In [ ]:
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
train = train.drop('cp_type', axis=1)

In [ ]:
train

In [ ]:
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test.drop('cp_type', axis=1)

In [ ]:
target = train[['sig_id']+target_cols]
target_ns = train[['sig_id']+target_nonsc_cols2]

In [ ]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()
target_cols

In [ ]:
b = train_targets_scored.columns[1:].tolist()

In [ ]:
target_cols == b

In [ ]:
n_comp1 = 50
n_comp2 = 15
num_features=len(feature_cols) + n_comp1 + n_comp2
num_targets=len(target_cols)
num_targets_0=len(target_nonsc_cols2)

In [ ]:
tar_freq = np.array([np.min(list(g_table(train[target_cols].iloc[:,i]).values())) for i in range(len(target_cols))])

In [ ]:
tar_weight0 = np.array([np.log(i+100) for i in tar_freq])

In [ ]:
np.min(tar_weight0)

In [ ]:
tar_weight0_min = dp(np.min(tar_weight0))

In [ ]:
tar_weight = tar_weight0_min/tar_weight0

In [ ]:
pos_weight = torch.tensor(tar_weight).to(DEVICE)

In [ ]:
from torch.nn.modules.loss import _WeightedLoss
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight,
                                                  pos_weight = pos_weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
class TrainDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return (self.features.shape[0])

    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct

class TestDataset:
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return (self.features.shape[0])

    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0

    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets) #with sigmoid
        loss.backward()
        optimizer.step()
        scheduler.step()

        final_loss += loss.item()

    final_loss /= len(dataloader)

    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []

    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets) #contains sigmoid

        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy()) #need sigmoid

    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)

    return final_loss, valid_preds

In [ ]:
def inference_fn(model, dataloader, device):
    model.eval()
    preds = []

    for data in dataloader:
        inputs = data['x'].to(device)
        with torch.no_grad():
            outputs = model(inputs)

        preds.append(outputs.sigmoid().detach().cpu().numpy())#need sigmoid

    preds = np.concatenate(preds)

    return preds

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        cha_1 = 256
        cha_2 = 512
        cha_3 = 512

        cha_1_reshape = int(hidden_size/cha_1)
        cha_po_1 = int(hidden_size/cha_1/2)
        cha_po_2 = int(hidden_size/cha_1/2/2) * cha_3

        self.cha_1 = cha_1
        self.cha_2 = cha_2
        self.cha_3 = cha_3
        self.cha_1_reshape = cha_1_reshape
        self.cha_po_1 = cha_po_1
        self.cha_po_2 = cha_po_2

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

        self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
        self.dropout_c1 = nn.Dropout(0.1)
        self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1,cha_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = cha_po_1)

        self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2 = nn.Dropout(0.1)
        self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

        self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_1 = nn.Dropout(0.3)
        self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

        self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_2 = nn.Dropout(0.2)
        self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_3, kernel_size = 5, stride = 1, padding=2, bias=True),dim=None)

        self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

    def forward(self, x):

        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.celu(self.dense1(x), alpha=0.06)

        x = x.reshape(x.shape[0],self.cha_1,
                      self.cha_1_reshape)

        x = self.batch_norm_c1(x)
        x = self.dropout_c1(x)
        x = F.relu(self.conv1(x))

        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = F.relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c2_1(x)
        x = self.dropout_c2_1(x)
        x = F.relu(self.conv2_1(x))

        x = self.batch_norm_c2_2(x)
        x = self.dropout_c2_2(x)
        x = F.relu(self.conv2_2(x))
        x =  x * x_s

        x = self.max_po_c2(x)

        x = self.flt(x)

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)

        return x

In [ ]:
len(train)*0.8

In [ ]:
train_df = train.iloc[:int(len(train)*0.8),:]
valid_df = train.iloc[int(len(train)*0.8):,:].reset_index(drop=True)

In [ ]:
valid_df

In [ ]:
x_train, y_train,y_train_ns = train_df[feature_cols], train_df[target_cols].values,train_df[target_nonsc_cols2].values
x_valid, y_valid,y_valid_ns  =  valid_df[feature_cols], valid_df[target_cols].values,valid_df[target_nonsc_cols2].values
x_test = test[feature_cols]

In [ ]:
col_num = list(set(feat_dic['gene'] + feat_dic['cell']) & set(feature_cols))

In [ ]:
x_train[col_num],ss = norm_fit(x_train[col_num],True,'quan')
x_valid[col_num]    = norm_tra(x_valid[col_num],ss)
x_test[col_num]     = norm_tra(x_test[col_num],ss)

In [ ]:
x_train

In [ ]:
def pca_pre(tr,va,te,
            n_comp,feat_raw,feat_new):
    pca = PCA(n_components=n_comp, random_state=42)
    tr2 = pd.DataFrame(pca.fit_transform(tr[feat_raw]),columns=feat_new)
    va2 = pd.DataFrame(pca.transform(va[feat_raw]),columns=feat_new)
    te2 = pd.DataFrame(pca.transform(te[feat_raw]),columns=feat_new)
    return(tr2,va2,te2)

In [ ]:
pca_feat_g = [f'pca_G-{i}' for i in range(n_comp1)]
feat_dic['pca_g'] = pca_feat_g
x_tr_g_pca,x_va_g_pca,x_te_g_pca = pca_pre(x_train,x_valid,x_test,
                                           n_comp1,feat_dic['gene'],pca_feat_g)


In [ ]:
x_train = pd.concat([x_train,x_tr_g_pca],axis = 1)
x_valid = pd.concat([x_valid,x_va_g_pca],axis = 1)
x_test  = pd.concat([x_test,x_te_g_pca],axis = 1)

In [ ]:
pca_feat_g = [f'pca_C-{i}' for i in range(n_comp2)]
feat_dic['pca_c'] = pca_feat_g
x_tr_c_pca,x_va_c_pca,x_te_c_pca = pca_pre(x_train,x_valid,x_test,
                                           n_comp2,feat_dic['cell'],pca_feat_g)
x_train = pd.concat([x_train,x_tr_c_pca],axis = 1)
x_valid = pd.concat([x_valid,x_va_c_pca],axis = 1)
x_test  = pd.concat([x_test,x_te_c_pca], axis = 1)

In [ ]:
x_train # features + 50 PCA-gene + 15 PCA-cell

In [ ]:
x_train,x_valid,x_test = x_train.values,x_valid.values,x_test.values

In [ ]:
x_train.shape

In [ ]:
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

In [ ]:
train_dataset = TrainDataset(x_train, y_train_ns)
valid_dataset = TrainDataset(x_valid, y_valid_ns)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
hidden_size=4096
model = Model(
    num_features=num_features,
    num_targets=num_targets_0,
    hidden_size=hidden_size,
)

model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=0.0001)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, 
                                          max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(trainloader))

loss_tr = nn.BCEWithLogitsLoss()   #SmoothBCEwLogits(smoothing = 0.001)
loss_va = nn.BCEWithLogitsLoss()    

early_stopping_steps = EARLY_STOPPING_STEPS
early_step = 0

In [ ]:
for epoch in range(1):
    train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
    valid_loss, valid_preds = valid_fn(model, loss_va, validloader, DEVICE)
    print(f"EPOCH: {epoch},train_loss: {train_loss}, valid_loss: {valid_loss}")

In [ ]:
model.dense3 = nn.utils.weight_norm(nn.Linear(model.cha_po_2, num_targets))
model.to(DEVICE)

train_dataset = TrainDataset(x_train, y_train)
valid_dataset = TrainDataset(x_valid, y_valid)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
trn_idx = train_df.index
val_idx = valid_df.index

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                          max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))

loss_tr = SmoothBCEwLogits(smoothing = 0.001) #the loss has pos_weight
loss_va = nn.BCEWithLogitsLoss()    #-log(sigmoid())

early_stopping_steps = EARLY_STOPPING_STEPS
early_step = 0

oof = np.zeros((len(train), len(target_cols)))
best_loss = np.inf

mod_name = f"FOLD_mod11_.pth"

In [ ]:
for epoch in range(EPOCHS):

    train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
    valid_loss, valid_preds = valid_fn(model, loss_va, validloader, DEVICE)
    print(f"EPOCH: {epoch},train_loss: {train_loss}, valid_loss: {valid_loss}")

    if valid_loss < best_loss:

        best_loss = valid_loss
        oof[val_idx] = valid_preds
        torch.save(model.state_dict(), mod_name)

    elif(EARLY_STOP == True):

        early_step += 1
        if (early_step >= early_stopping_steps):
            break

In [ ]:
testdataset = TestDataset(x_test)
testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)

model = Model(
    num_features=num_features,
    num_targets=num_targets,
    hidden_size=hidden_size,
)

model.load_state_dict(torch.load(mod_name))
model.to(DEVICE)

#predictions = np.zeros((len(test_), len(target_cols)))
predictions = inference_fn(model, testloader, DEVICE)

In [ ]:
test[target_cols] = predictions

In [ ]:
test

In [ ]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
sub